# Ejercicio 3: Método de la composición
**a)** Suponga que es relativamente fácil generar $n$ variables aleatorias a partir de sus distribuciones de probabilidad $ F_i $, $i = 1, \ldots, n $. Implemente un método para generar una variable aleatoria cuya distribución de probabilidad es:


$$
\displaystyle F(x) = \sum_{i=1}^{n} p_i \cdot F_i(x) 
$$

donde $p_i$ son números no negativos cuya suma es 1.

In [8]:
#Importaciones
from typing import Callable
from itertools import accumulate
from random import random
from math import log

In [31]:
def Fs_generator(accumulate_ps:list[float], Fs:list[Callable[[float], float]]) -> Callable[[float],float]:
    """
    Generador de funciones de distribución acumulada

    Args:
        accumulate_ps (list[float]): Lista de valores acumulados
        Fs (list[Callable[[float], float]]): Lista de funciones de distribución acumulada

    Returns:
        Callable[[float],float]: Función de distribución acumulada
    """
    U = random()
    for i in range(len(accumulate_ps)):
        if U <= accumulate_ps[i]:
            return Fs[i]()
    return Fs[-1]() 


from random import random
from math import log
from itertools import accumulate
from typing import Callable, List

def Fs_generator(accumulate_ps: List[float], Fs: List[Callable[[], float]]) -> Callable[[], float]:
    """
    Generador de variables aleatorias por composición.

    Args:
        accumulate_ps (List[float]): Probabilidades acumuladas.
        Fs (List[Callable[[], float]]): Lista de generadores de variables aleatorias.

    Returns:
        Callable[[], float]: Función que genera valores según la mezcla.
    """
    def generar_valor():
        U = random()
        for i, p_acum in enumerate(accumulate_ps):
            if U <= p_acum:
                return Fs[i]()
        return Fs[-1]()  # Por si hay errores de redondeo
    return generar_valor  # Devuelve una función, no un valor

def F(Fs: List[Callable[[], float]], ps: List[float]) -> Callable[[], float]:
    """
    Función de distribución compuesta por método de composición.

    Args:
        Fs (List[Callable[[], float]]): Generadores de variables aleatorias.
        ps (List[float]): Probabilidades de cada componente.

    Returns:
        Callable[[], float]: Generador compuesto.
    """
    assert all(p > 0 for p in ps), "Probabilidades deben ser positivas."
    assert abs(sum(ps) - 1) < 1e-9, "Las probabilidades deben sumar 1."
    assert len(Fs) == len(ps), "Debe haber una probabilidad por cada distribución."

    accumulate_ps = list(accumulate(ps))
    return Fs_generator(accumulate_ps, Fs)

**b)** Genere datos usando tres exponenciales independientes con media $3$, $5$ y $7$ respectivamente y $p =
(0.5,0.3,0.2)$. Calcule la esperanza exacta de la mezcla y estime con 10.000 repeticiones. Tenga
cuidado con la parametrización que este usando!!

In [ ]:
def Exponential(lamda:float) -> float:
    """
    Variable aleatoria exponencial con parámetro lambda

    Args:
        lamda (float): Parámetro de la distribución

    Returns:
        float: Valor aleatorio con distribución exponencial
    """
    return -log(1-random()) / lamda


def theorical_hope(ps: List[float], lamdas: List[float]) -> float:
    """
    Calcula la esperanza exacta de una mezcla de exponenciales.

    Args:
        ps (List[float]): Probabilidades de mezcla.
        lamdas (List[float]): Parámetros de las exponenciales.

    Returns:
        float: Esperanza teórica.
    """
    return sum(p / lamda for p, lamda in zip(ps, lamdas))

def hope_estimation(samples: List[float]) -> float:
    """Estimación de la esperanza por Monte Carlo."""
    return sum(samples) / len(samples)

In [60]:
ps = [0.5, 0.3, 0.2]
lamdas = [1/3, 1/5, 1/7]
Nsim = 10_000
Fs = [lambda lamda=lamda: Exponential(lamda) for lamda in lamdas]

# F es un generador compuesto
Z = F(Fs=Fs, ps=ps)

# Generamos muestras
samples = [Z() for _ in range(Nsim)]

print(f"E[F] = {theorical_hope(ps=ps, lamdas=lamdas)}")
print(f"  θ  ≈ {hope_estimation(samples=samples):3f}")


E[F] = 4.4
  θ  ≈ 4.370477
